In [45]:
# -*- coding: utf-8 -*-
"""
Yahoo + FRED ➜ ZeroCurve ➜ Dell Corp 5.40 % (10‑sep‑2040)

Requisitos:
    pip install yfinance pandas_datareader QuantLib-Python
"""

# ───────────────── IMPORTS ─────────────────
from datetime import datetime
import yfinance as yf
from pandas_datareader import data as web
import QuantLib as ql

# ───────────────── 1. RENDIMIENTOS ─────────────────
YAHOO_TICS = {         # tenor ➜ ticker
    "3M":  "^IRX",     # 13‑week T‑Bill
    "5Y":  "^FVX",
    "10Y": "^TNX",
    "30Y": "^TYX",
}

rates = {}             # tenor ➜ float (decimal)

for tenor, tic in YAHOO_TICS.items():
    raw = yf.download(tic, period="6mo", interval="1d", progress=False)
    if raw.empty:
        print(f"⚠  {tic} sin datos; omitido.");  continue
    col = "Adj Close" if "Adj Close" in raw.columns else "Close"
    last = raw[col].dropna().iloc[-1].item()          # float
    rates[tenor] = last / 100.0                       # % ➜ decimal

# 2‑años FRED (DGS2) si falta
if "2Y" not in rates:
    try:
        fred = web.DataReader("DGS2", "fred",
                              start=datetime.now().replace(year=datetime.now().year - 1))
        rates["2Y"] = fred.dropna().iloc[-1].item() / 100.0
        print("✓  Añadido nodo 2Y desde FRED (DGS2).")
    except Exception as e:
        print("⚠  DGS2 no disponible; curva sin 2Y.")

if len(rates) < 3:
    raise SystemExit("Demasiados nodos faltantes; abortando.")

print("\nRendimientos spot")
for t, r in sorted(rates.items(), key=lambda x: ("M" in x[0], int(x[0][:-1]))):
    print(f"{t:>3}: {r*100:.3f} %")

# ───────────────── 2. CURVA ZERO ─────────────────
CAL = ql.UnitedStates(ql.UnitedStates.GovernmentBond)
DC  = ql.ActualActual(ql.ActualActual.ISDA)
today = ql.Date.todaysDate()
ql.Settings.instance().evaluationDate = today

def tenor_to_date(t):
    n = int(t[:-1])
    return today + (ql.Period(n, ql.Months) if t.endswith("M")
                    else ql.Period(n, ql.Years))

nodes = [(today, 0.0)] + sorted(
    [(tenor_to_date(k), v) for k, v in rates.items()],
    key=lambda x: x[0])

dates, ys = zip(*nodes)

zc = ql.ZeroCurve(list(dates), list(ys), DC, CAL)
zc.enableExtrapolation()
curve = ql.YieldTermStructureHandle(zc)

# ───────────────── 3. BONO DELL 5.40 % 2040 ─────────────────
coupon   = 0.054
issue    = ql.Date(10,  9, 2010)
maturity = ql.Date(10,  9, 2040)

sched = ql.Schedule(issue, maturity, ql.Period(ql.Semiannual),
                    CAL, ql.Following, ql.Following,
                    ql.DateGeneration.Backward, False)

bond = ql.FixedRateBond(2, 100, sched, [coupon], DC)
bond.setPricingEngine(ql.DiscountingBondEngine(curve))

clean_px = bond.cleanPrice()

# ——— YTM robusto (yieldRate, yield o Brent) ———
if hasattr(ql.BondFunctions, "yieldRate"):
    ytm_dec = ql.BondFunctions.yieldRate(bond, clean_px, DC,
                                         ql.Compounded, ql.Semiannual)
elif hasattr(ql.BondFunctions, "yield"):
    ytm_dec = getattr(ql.BondFunctions, "yield")(bond, clean_px, DC,
                                                 ql.Compounded, ql.Semiannual)
else:                              # fallback numérico
    solver = ql.Brent()
    f = lambda y: ql.BondFunctions.cleanPrice(
        bond, y, DC, ql.Compounded, ql.Semiannual) - clean_px
    ytm_dec = solver.solve(f, 1e-12, 0.05, 1e-6, 1.0)

ytm_pct = ytm_dec * 100

# DV01 y convexidad con la misma TIR
dv01 = ql.BondFunctions.basisPointValue(
           bond, ytm_dec, DC, ql.Compounded, ql.Semiannual) / 10000.0
conv = ql.BondFunctions.convexity(
           bond, ytm_dec, DC, ql.Compounded, ql.Semiannual)

# ───────────────── 4. RESULTADOS ─────────────────
print("\n—— Valuación Dell 5.40 % 10‑sep‑2040 ——")
print(f"Precio limpio : {clean_px:,.3f}")
print(f"TIR (YTM)     : {ytm_pct:,.3f} %")
print(f"DV01          : {dv01:,.4f}")
print(f"Convexidad    : {conv:,.4f}")



✓  Añadido nodo 2Y desde FRED (DGS2).

Rendimientos spot
 2Y: 3.910 %
 5Y: 3.977 %
10Y: 4.420 %
30Y: 4.964 %
 3M: 4.235 %

—— Valuación Dell 5.40 % 10‑sep‑2040 ——
Precio limpio : 109.466
TIR (YTM)     : 4.528 %
DV01          : -0.0000
Convexidad    : 138.2881
